In [3]:
import pandas as pd
import numpy as np

In [14]:
pd.set_option('display.max_columns', None)

In [40]:
## Features to be used

# - Fatigue
#   Days since last match
#   Distance covered in the last x matches?

# - Home Team Form
#   Goals difference of home team in the last x matches    
#   Goals difference of home team in the last x home matches    
#   Average number of points gained by home team in the last x matches
#   Number of home matches won by home team in its last x home matches
#   Home Team Win streak  
#   Home Team Newly Promoted Team?

# - Away Team Form
#   Goals difference of away team in the last x matches  
#   Goals difference of away team in the last x away matches
#   Average number of points gained by away team in the last x matches
#   Number of away matches won by away team in its last x away matches
#   Away Team Win streak
#   Away Team Newly Promoted Team?

# - Home Team Performance Index
#   Home Defense Performance Index
#   Home Midfield Performance Index
#   Home Attack Performance Index

# - Away Team Performance Index
#   Away Defense Performance Index
#   Away Midfield Performance Index
#   Away Attack Performance Index

# - Betting Odds
#     B365H
#     B365D
#     B365A

# - Head to Head
#   Goals difference in previous x encounters

## Data Preparation

In [36]:
df = pd.DataFrame()

In [38]:
# read csv file for match statistics
match_statistics_17_18_df = pd.read_csv('datasets/2017-2018/2017-2018-match-statistics.csv')

# drop irrelvant columns
match_statistics_17_18_df.drop(match_statistics_17_18_df.loc[:, 'BWH':].columns, axis=1, inplace = True)
match_statistics_17_18_df.drop(['Div', 'HTHG', 'HTAG', 'HTR', 'Referee'], axis=1, inplace = True)

match_statistics_17_18_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,11/08/2017,Arsenal,Leicester,4,3,H,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.5,6.50
1,12/08/2017,Brighton,Man City,0,2,A,6,14,2,4,6,9,3,10,0,2,0,0,11.00,5.5,1.33
2,12/08/2017,Chelsea,Burnley,2,3,A,19,10,6,5,16,11,8,5,3,3,2,0,1.25,6.5,15.00
3,12/08/2017,Crystal Palace,Huddersfield,0,3,A,14,8,4,6,7,19,12,9,1,3,0,0,1.83,3.6,5.00
4,12/08/2017,Everton,Stoke,1,0,H,9,9,4,1,13,10,6,7,1,1,0,0,1.70,3.8,5.75


In [39]:
# read csv file for each team
arsenal_fixtures_17_18_df = pd.read_csv('datasets/2017-2018/arsenal-fixtures.csv')

# drop irrelvant columns
arsenal_fixtures_17_18_df.drop(['Time', 'Day', 'Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 'Notes'], axis=1, inplace = True)

# add new feature: 'Days Since Last Match'
# filter by home matches only
# add new feature: 'HomeTeam'
# add new feature: 'APoss'
# change feature name: {'xG': 'HxG', 'xGA': 'AxG', 'Poss': 'HPoss', 'Opponent': 'AwayTeam'}

arsenal_fixtures_17_18_df.head()

,Date,Comp,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss
0,2017-08-06,Community Shield,FA Community Shield,Neutral,D,1 (4),1 (1),Chelsea,NaN,NaN,NaN
1,2017-08-11,Premier League,Matchweek 1,Home,W,4,3,Leicester City,2.5,1.5,68.0
2,2017-08-19,Premier League,Matchweek 2,Away,L,0,1,Stoke City,1.5,0.7,76.0
3,2017-08-27,Premier League,Matchweek 3,Away,L,0,4,Liverpool,0.6,3.1,52.0
4,2017-09-09,Premier League,Matchweek 4,Home,W,3,0,Bournemouth,2.2,0.6,58.0
